# Data import

In [1]:
import psycopg2
import pandas as pd
import numpy as np

In [2]:

conn_string = "host='localhost' dbname='weather_env' user='postgres' password='postgres'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()
cur.execute("""SELECT * FROM environment_data.pivoted;""")
records = cur.fetchall()
cur.close()
df = pd.DataFrame(records, columns=[x[0] for x in cur.description])

In [3]:
ddf = df.describe().T
subdif = .5
cols = (ddf['count'] > np.max(ddf['count'].values) * subdif).replace(False, np.nan).dropna().index
df = df[cols].dropna()
df.shape

(57738, 5)

In [4]:
import sys
sys.path.append(r'C:\Users\woute\Documents\GitHub\WaterEnvDataWarehouse\dags')
from common.windowing import WindowGenerator

In [5]:
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

# TF example

In [6]:
look_back, shift, look_fore = 7*2, 0, 1
wg = WindowGenerator(
    df,
    name=None,
    distribution=(look_back, shift, look_fore),
    delete_target=False,
    split=(.6, .3, .1),
    normalizer=True
)
wg()
wg.train[0].shape, df.columns.values

((34642, 14, 5),
 array(['measure_year', 'measure_month', 'measure_day',
        'lobith_waterlevel', 'arnhem_waterlevel'], dtype=object))

In [7]:
model = tf.keras.Sequential([
    # tf.keras.layers.LSTM(128, 'relu', return_sequences=True, input_shape=wg.train[0].shape[1:], dropout=.0, recurrent_dropout=.0,),
    # tf.keras.layers.Conv1D(32, activation='relu', kernel_size=(look_back, )),
    # tf.keras.layers.LSTM(128, 'relu', return_sequences=True),
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dropout(.5),
    # tf.keras.layers.Dense(128),
    # tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(look_fore*wg.col),
    tf.keras.layers.Reshape([look_fore, wg.col])
])

In [ ]:
metrics=[tf.metrics.MeanSquaredError()]

model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(), metrics=metrics)
    
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_mean_squared_error', patience=0)
model.fit(*wg.train, epochs=100, callbacks=[early_stopping], validation_data=wg.valid, verbose=0)

# model.summary()
model.evaluate(*wg.train)
model.evaluate(*wg.valid)
model.evaluate(*wg.testd)

In [13]:
from scikeras.wrappers import KerasRegressor

model.build(wg.train[0].shape[1:])

sk_model = KerasRegressor(
    model=model,
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.metrics.MeanSquaredError()],
    epochs=10
)
sk_model

KerasRegressor(
	model=<keras.engine.sequential.Sequential object at 0x0000025CBB44AFB0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=<keras.optimizers.optimizer_experimental.adam.Adam object at 0x0000025CBD0FC280>
	loss=<keras.losses.MeanSquaredError object at 0x0000025CBD0FFBB0>
	metrics=[<keras.metrics.metrics.MeanSquaredError object at 0x0000025CBD0FF550>]
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
)

In [14]:
from sklearn.model_selection import GridSearchCV

# 'model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size',
# 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs'
parameters = {
    'optimizers': ['adam', ], # 'rmsprop', ],
    'init':       ['glorot_uniform', 'normal', 'uniform'],
    'epochs':     [10, ],
    # 'batches':    [5, 10, 20],
}

gscv = GridSearchCV(
    sk_model,
    parameters,
    cv=10,
    scoring=('r2'),
)
gscv

GridSearchCV(cv=10,
             estimator=KerasRegressor(epochs=10, loss=<keras.losses.MeanSquaredError object at 0x0000025CBD0FFBB0>, metrics=[<keras.metrics.metrics.MeanSquaredError object at 0x0000025CBD0FF550>], model=<keras.engine.sequential.Sequential object at 0x0000025CBB44AFB0>, optimizer=<keras.optimizers.optimizer_experimental.adam.Adam object at 0x0000025CBD0FC280>),
             param_grid={'epochs': [10],
                         'init': ['glorot_uniform', 'normal', 'uniform'],
                         'optimizers': ['adam']},
             scoring='r2')

In [15]:
gscv.fit(*wg.train)

INFO:tensorflow:Assets written to: C:\Users\woute\AppData\Local\Temp\tmps1yr5djo\assets


AttributeError: 'NoneType' object has no attribute 'build'

In [ ]:
ref = (wg.testd[1].squeeze() * wg.feature_std + wg.feature_mean)[:, wg.col_num['arnhem_waterlevel']]
pred = (model.predict(wg.testd[0]).squeeze() * wg.feature_std + wg.feature_mean)[:, wg.col_num['arnhem_waterlevel']]

plt.plot(ref)
plt.plot(pred)
plt.ylim(800, 1000)
plt.show()

In [ ]:
plt.plot(wg.testd[1].squeeze()[:, wg.col_num['arnhem_waterlevel']][:150])

n = wg.col_num['arnhem_waterlevel']
p = wg.testd[0][[0]]

for x in range(150):
    new_p = p[:, 1:]
    pred = model.predict(p, verbose=0)
    p = np.concatenate([new_p, pred], axis=1)
    plt.plot([x], p[0, 0, n], '.r')

# plt.ylim(0, 1500)
plt.show()

# Classification

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

In [ ]:
df['moved'] = (df['arnhem_waterlevel'] < 800).astype(int)
df['arnhem_waterlevel'].count() / df['moved'].sum()

In [ ]:
look_back, shift, look_fore = 28, 0, 1
wg = WindowGenerator(
    df,
    name=None,
    distribution=(look_back, shift, look_fore),
    delete_target=False,
    split=(.7, .2, .1),
    shuffle=True
)
wg()
wg.train[0].shape, wg.train[0][0, 0]

In [ ]:
model = tf.keras.Sequential([
    # tf.keras.layers.Normalization(axis=(1,2)),
    tf.keras.layers.Conv1D(132, activation='relu', kernel_size=(look_back, )),
    tf.keras.layers.Dense(look_fore*wg.col, kernel_initializer=tf.initializers.zeros()),
    # tf.keras.layers.Dense(look_fore*wg.col*10),
    # tf.keras.layers.Dense(look_fore*wg.col*10),
    # tf.keras.layers.Dense(look_fore*wg.col*10),
    tf.keras.layers.Dense(look_fore*wg.col),
    tf.keras.layers.Reshape([look_fore, wg.col]),
])

In [ ]:

model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=[tf.keras.metrics.Accuracy()])
    
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
model.fit(*wg.train, epochs=100, callbacks=[early_stopping], validation_data=wg.valid)#, verbose=0)

model.summary()

In [ ]:
model.evaluate(*wg.train)
model.evaluate(*wg.valid)
model.evaluate(*wg.testd)

plt.plot(wg.testd[1].squeeze()[:, wg.col_num['moved']])
plt.plot(model.predict(wg.testd[0]).squeeze()[:, wg.col_num['moved']])
plt.ylim(-1, 2)
plt.show()